In [87]:
import openpyxl as xl
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from datetime import datetime
from typing import Sequence, Literal, Dict, Any
import math

In [88]:

def ttest_two_groups(
    group1: Sequence[float],
    group2: Sequence[float],
    *,
    equal_var: bool | None = None,
    transform: Literal['none', 'logit'] = 'none',
    clip_eps: float = 1e-6,
    return_all: bool = True
) -> Dict[str, Any]:
    """
    2群の t 検定と効果量を計算するユーティリティ関数。
    
    Parameters
    ----------
    group1, group2 : Sequence[float]
        数値リスト（list, tuple, np.array）。None/NaN は除外。
    equal_var : bool | None
        True なら Student, False なら Welch。
        None の場合は Levene 検定 (p>=0.05) なら True, そうでなければ False を自動選択。
    transform : {'none','logit'}
        'logit' を指定すると比率(0〜1)に微小クリップした上で log(p/(1-p)) へ変換して検定。
    clip_eps : float
        logit 変換時の端点クリップ閾値。0,1 を避けるため [clip_eps, 1-clip_eps] へ収める。
    return_all : bool
        True なら補助統計（分散, 標準誤差など）も返す。
    
    Returns
    -------
    result : dict
        主キー: 
          'test' ('student' or 'welch'),
          't', 'df', 'p',
          'mean1','mean2','mean_diff',
          'ci_diff' (95%CI tuple),
          'cohens_d','hedges_g','g_ci' (95%CI),
          'glass_delta',
          'levene_p'
        transform='logit' の場合は 'inverse_means' (原スケール近似) を付加。
    """
    def _clean(x):
        arr = np.array([v for v in x if v is not None], dtype=float)
        arr = arr[~np.isnan(arr)]
        return arr

    g1 = _clean(group1)
    g2 = _clean(group2)
    if g1.size < 2 or g2.size < 2:
        raise ValueError("各群には少なくとも2つ以上の有効な数値が必要です。")

    original_means = (g1.mean(), g2.mean())

    if transform == 'logit':
        g1 = np.clip(g1, clip_eps, 1 - clip_eps)
        g2 = np.clip(g2, clip_eps, 1 - clip_eps)
        g1 = np.log(g1 / (1 - g1))
        g2 = np.log(g2 / (1 - g2))

    n1, n2 = g1.size, g2.size
    mean1, mean2 = g1.mean(), g2.mean()
    var1, var2 = g1.var(ddof=1), g2.var(ddof=1)

    # Levene で等分散性判定（center='mean'）
    levene_stat, levene_p = stats.levene(g1, g2, center='mean')

    # equal_var 自動決定
    if equal_var is None:
        equal_var = (levene_p >= 0.05)

    # t 検定
    t_stat, p_val = stats.ttest_ind(g1, g2, equal_var=equal_var)

    if equal_var:
        df = n1 + n2 - 2
        test_type = 'student'
        # プール分散
        sp2 = ((n1 - 1)*var1 + (n2 - 1)*var2) / (n1 + n2 - 2)
        se_diff = math.sqrt(sp2 * (1/n1 + 1/n2))
    else:
        # Welch の自由度
        df = (var1/n1 + var2/n2)**2 / ((var1**2)/(n1**2*(n1-1)) + (var2**2)/(n2**2*(n2-1)))
        test_type = 'welch'
        se_diff = math.sqrt(var1/n1 + var2/n2)

    mean_diff = mean1 - mean2
    t_crit = stats.t.ppf(0.975, df)
    ci_diff = (mean_diff - t_crit*se_diff, mean_diff + t_crit*se_diff)

    # 効果量
    # プール標準偏差（等分散仮定用）
    sp = math.sqrt(((n1 - 1)*var1 + (n2 - 1)*var2) / (n1 + n2 - 2))
    cohens_d = (mean1 - mean2) / sp

    # Hedges g 補正
    J = 1 - 3/(4*(n1 + n2) - 9)
    hedges_g = cohens_d * J

    # g の SE (近似) & CI
    se_g = math.sqrt((n1 + n2)/(n1*n2) + (hedges_g**2)/(2*(n1 + n2 - 2)))
    g_ci = (hedges_g - t_crit*se_g, hedges_g + t_crit*se_g)

    # Glass Δ (第2群を基準：分散異質時の代替)
    glass_delta = (mean1 - mean2) / math.sqrt(var2)

    result = {
        'test': test_type,
        'equal_var_assumed': equal_var,
        't': t_stat,
        'df': df,
        'p': p_val,
        'mean1': mean1,
        'mean2': mean2,
        'mean_diff': mean_diff,
        'ci_diff': ci_diff,
        'cohens_d': cohens_d,
        'hedges_g': hedges_g,
        'g_ci': g_ci,
        'glass_delta': glass_delta,
        'n1': n1,
        'n2': n2,
        'var1': var1,
        'var2': var2,
        'levene_p': levene_p,
        'transform': transform,
    }

    if transform == 'logit':
        # 参考：逆 logit で平均（logit平均）を原スケールへ近似
        inv = lambda x: 1 / (1 + math.exp(-x))
        result['inverse_means'] = (inv(mean1), inv(mean2))

    if not return_all:
        # 最小限
        keys = ['test','t','df','p','mean1','mean2','mean_diff','ci_diff','hedges_g','g_ci']
        result = {k: result[k] for k in keys}
    return result


In [89]:
def run_anova_from_lists(group_data: dict,pjt, filename_prefix="anova_result"):
    """
    group_data: dict
        キーがグループ名、値が数値リストの辞書
        例: {
            'high': [320, 310, 300],
            'mid': [280, 270, 290],
            'low': [250, 240, 260]
        }
    filename_prefix: str
        出力ファイルのプレフィックス（デフォルト: "anova_result"）
    """

    # --- データ整形 ---
    data = []
    for group, values in group_data.items():
        for val in values:
            data.append({'group': group, 'fixation': val})

    df = pd.DataFrame(data)

    # --- 一元配置分散分析 (ANOVA) ---
    model = ols('fixation ~ group', data=df).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)

    # --- Tukey事後比較 ---
    tukey = pairwise_tukeyhsd(endog=df['fixation'], groups=df['group'], alpha=0.05)

    # --- 結果保存 ---
    filename_prefix = "1_anova_Tukey"
    filename = f"{filename_prefix}_log.txt"  # 例: anova_result_log.txt

    with open(filename, "a", encoding="utf-8") as f:
        f.write(f"pjt: {pjt}")
        f.write(f"\n==== 実行日時: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} ====\n")
        f.write("=== 一元配置分散分析（ANOVA）結果 ===\n")
        f.write(anova_table.to_string())
        f.write("\n\n=== Tukey HSD 多重比較結果 ===\n")
        f.write(str(tukey.summary()))
        f.write("\n" + "="*50 + "\n")

    print(f"✅ 結果を保存しました: {filename}")
    return anova_table, tukey

In [ ]:
wb = xl.load_workbook("src_.xlsx")
sheet_input = wb["Tot Fixation dur incl 0"]
sheet_pjt_aoi = wb["aoi_解析"]

# 高頻度のセグメントでの分析(1が高頻度)

In [91]:
past_pjt = ""
pjt_list =[]
aoi_list = []
for row in range(2,422):
    if sheet_pjt_aoi.cell(row=row, column=1).value not in pjt_list:
        pjt_list.append(sheet_pjt_aoi.cell(row=row, column=1).value)
    if sheet_pjt_aoi.cell(row=row, column=2).value not in aoi_list:
        aoi_list.append(sheet_pjt_aoi.cell(row=row, column=2).value)

data ={}

for pjt in pjt_list:
    if pjt not in data or data[pjt] is None:
        data[pjt] = {}
    for aoi in aoi_list:
        data[pjt][aoi] = []

In [92]:
for row in range(2,422):
    pjt = sheet_pjt_aoi.cell(row=row, column=1).value
    aoi_name = sheet_pjt_aoi.cell(row=row, column=2).value
    #^ 1の方
    data[pjt][aoi_name].append(sheet_pjt_aoi.cell(row=row, column=5).value)


In [93]:
for pjt in pjt_list:
    for aoi in aoi_list:
        if len(data[pjt][aoi]) <= 0:
            del data[pjt][aoi]

del data["all"]

In [94]:
pjt_list.remove("all")

In [95]:

for pjt in pjt_list:
    print(data[pjt])
    anova_table, tukey = run_anova_from_lists(data[pjt],pjt)
    tukey_df = pd.DataFrame(
        tukey._results_table.data[1:],  # 1行目はヘッダ
        columns=tukey._results_table.data[0]
    )
    tukey_df[tukey_df['reject'] == True].to_csv(f"t-test/t-test_rejectTrue_{pjt}.csv", index=False)

{'女L': [0, 0, 0.03529565457750602, 0.0933673345193884, 0.04070771037881593], '女R': [0, 0.09928623481515325, 0, 0, 0], '女イヤリング\u3000顔': [0.9406242213901455, 0.4790562126831407, 0.5580941668850623, 0.5633571426303542, 0.19872951325494], '女イヤリング\u3000髪型': [0, 0.008628769817731688, 0, 0, 0.007072954304610395]}
✅ 結果を保存しました: 1_anova_Tukey_log.txt
{'女all \u3000顔': [0.8937792378248078, 0.6802261147022172, 0.8829409277063418, 0.6609707331311674, 0.4488991651368647], '女all ネックレス': [0, 0, 0, 0, 0.003743138851220979], '女all 髪型': [0, 0.05513557467050272, 0, 0, 0.01966915707790071], '女allL': [0, 0, 0, 0.01017557676153663, 0.1153060082074882], '女alllR': [0, 0.037175818062278, 0, 0.01861530365910997, 0.01500523525576688]}
✅ 結果を保存しました: 1_anova_Tukey_log.txt
{'Rectangle': [0, 0, 0, 0, 0], '女無し L': [0, 0, 0, 0.01056949175132425, 0], '女無し\u3000顔': [0.9103276644778316, 0.3933015226823312, 0.7942053132931836, 0.7535022268558357, 0.0801662123060243], '女無しR': [0, 0, 0, 0, 0], '女無しネックレス': [0, 0, 0, 0, 0]}
✅ 結果